#### Section 1: EDA Imports and Methods
This EDA focused on time series and cross-sectional data. This EDA
initially provides summary statistics, then does a test to see if there's a time series pattern.
Then, if a dataset appears to have missing values, it imputes them. Regardless of whether or not
the data is time series or not, it calculates feature importance. Finally, if it is a time series,
it provides tests to determine the quality of the time series.

In [1]:
import os
import numpy as np
import pandas as pd
pd.set_option('display.width', 128)
pd.set_option('display.expand_frame_repr', True)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 20)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
# from fitter import Fitter, get_common_distributions, get_distributions
from pandas.plotting import autocorrelation_plot
from pmdarima.arima import ADFTest
from pmdarima.arima import auto_arima
# from rfpimp import *
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
# from clusteval import clusteval
from sklearn.model_selection import GridSearchCV, KFold, train_test_split, TimeSeriesSplit
from sklearn.metrics import r2_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.impute import IterativeImputer
from sklearn import linear_model
from statsmodels.tsa.stattools import adfuller

from ydata_profiling import ProfileReport as ydata_prof
from pandas_profiling import ProfileReport as pd_prof

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=ValueError)

ModuleNotFoundError: No module named 'pmdarima'

In [ ]:
os.chdir('C:/Users/norri/Desktop/RW5/')

In [ ]:
df = pd.read_csv('C:/Users/norri/Desktop/RW5/ICRForm.csv')
# df_date = pd.to_datetime(df['DATE'])

The following definitions are mostly used for descriptive statistics, but also provide
feature importance metrics towards the end.

In [ ]:
def missing_values(df):
    """

    This function is used early on in the EDA to determine how manu
    are missing.
    """
    names = [var for var in df.columns]
    missing_count = df[names].isnull().sum()
    var_count = np.array(df[names].isnull().sum() * 100/ len(df)).round(2)
    missing = pd.DataFrame(index=names)
    missing["Count Missing"] = missing_count
    missing["Percent Missing"] = var_count
    print(missing)


def unique(df):
    """

    Like the above method, it calculates the number of unique entries.
    """
    percent_unique = np.array(100 * df.nunique()/len(df.index)).round(2)
    count_unique = df.nunique()
    names = [var for var in df.columns]
    unique_df = pd.DataFrame(index=names)
    unique_df["Count Unique"] = count_unique
    unique_df["Percent Unique"] = percent_unique
    print(unique_df)


def corr_plot(df):
    """

    This provides a clean heatmap of correlations, though if too
    many variables are in the dataset, it will b edifficult to read
    """
    corr_temp = df.drop(['DATE'], axis=1)
    corr_names = corr_temp.columns.tolist()
    temp_df = df[corr_names]
    corr = temp_df.corr(method="pearson").round(2)
    mask = np.triu(np.ones_like(corr, dtype=bool))
    f, ax = plt.subplots(figsize=(8, 8))
    cmap = sns.diverging_palette(250, 1, as_cmap=True)
    sns.heatmap(corr, annot=True, mask=mask, cmap=cmap,
                vmax=1, vmin=-1, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})

def r2(rf, X_train, y_train):
    return r2_score(y_train, rf.predict(X_train))


def distributions(df):
    """

    This method tests the most common distributions and
    keeps the two, as well as plotting the distribution
    against a histogram of the data
    """
    for var in temp_df:
        dist_test = temp_df[var].dropna()
        dist_test = dist_test.values
        f = Fitter(dist_test, distributions='common', bins=100, timeout=30)
        f.fit()
        print(var)
        print(f.summary(Nbest=2, clf=True, plot=True))
        print(f.get_best(method='sumsquare_error'))


def summary(df):
    """

    This method simply pulls in three of the above methods
    and reports them at once.
    """
    print(missing_values(df))
    print(unique(df))
    corr_plot(df)


def forecast_accuracy(forecast, Actuals):
    """

    Calculates several accuracy metrics in the time series section
    """
    mape = np.mean(np.abs(forecast - Actuals)/np.abs(Actuals))  # MAPE
    mae = np.mean(np.abs(forecast - Actuals))    # MAE
    mse = np.square(np.subtract(Actuals,forecast)).mean()
    rmse = np.mean((forecast - Actuals)**2)**.5  # RMSE
    smape = 1/len(Actuals) * np.sum(2 * np.abs(forecast-Actuals) / (np.abs(Actuals) + np.abs(forecast))*100)
    return {'MAPE':mape, 'MSE':mse, 'MAE':mae, 'RMSE':rmse, 'SMAPE':smape}


def plot_df(df, x, y, title="", xlabel='DATE', ylabel='revenue', dpi=100):
    """

    A plot that demonstrates a dataset may be a time series plot.
    """
    plt.figure(figsize=(12, 4), dpi=dpi)
    plt.plot(x, y, color='blue')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

#### Section 2: Descriptive Statistics
This section provides descriptive statistics on the original dataframe. Assuming
some data is missing, not all the tests may work. Additionally, a basic time series
plot gives insight into whether or not the data is time-based in nature.

In [ ]:
df.describe()

In [ ]:
summary(df)

This plot can give us a good initial ideal that there is a time series.

In [ ]:
plot_df(df, df['DATE'], df['revenue'], title='Sales Over Time')

### Section 3: Prep for future analyses

In [ ]:
df = df[['DATE', 'revenue', 'display_S', 'b_branded_S', 'b_category_S',
         'p_auto_S', 'p_brand_S', 'p_category_S', 'p_competitive_S']]
temp_week = df['DATE']
corr_temp = df.drop(['DATE'], axis=1)
corr_names = corr_temp.columns.tolist()

### Section 4: Distribution Analysis and Descriptive Statistics
To compare the original dataset and the imputations, each data set finds
the top two distributions for every variable and its parameters. For imputations
that are necessary for a large number of missing variables, checking this section
and the next are crucial.

In [ ]:
temp_df = df.drop(['DATE'], axis=1)

for var in temp_df:
    dist_test = temp_df[var].dropna()
    dist_test = dist_test.values
    f = Fitter(dist_test, distributions='common', bins=100, timeout=30)
    f.fit()
    print(var)
    print(f.summary(Nbest=2, clf=True, plot=True))
    print(f.get_best(method='sumsquare_error'))

In [ ]:
df.describe()

### Section 5: KNN and MICE Imputation
This section uses the KNN and MICE methods of imputation. They are generally
the most accurate of methods, though one or the other may be more accurate
depending on the situation. It is important to note that if a data set with
no missing values is run through the EDA, no differences will be noted in
this section.

In [ ]:
knn_df_names = tuple(corr_names)
knn_temp = df[corr_names]
df_knn = knn_temp.filter(knn_df_names, axis=1).copy()

In [ ]:
knn_df_names = tuple(corr_names)
knn_temp = df[corr_names]
df_knn = knn_temp.filter(knn_df_names, axis=1).copy()
scaler = MinMaxScaler(feature_range=(0, 1))
df_knn = pd.DataFrame(scaler.fit_transform(df_knn), columns = df_knn.columns)

In [ ]:
knn_imputer = KNNImputer(n_neighbors=5, weights='distance', metric='nan_euclidean')
df_knn_imputed = pd.DataFrame(knn_imputer.fit_transform(df_knn), columns=df_knn.columns)
df_knn_imputed = pd.DataFrame(scaler.inverse_transform(df_knn_imputed), columns=df_knn.columns)
KNN_imputation = pd.concat([df_knn_imputed, temp_week], axis=1)

In [ ]:
mice_names = tuple(corr_names)
mice_temp = df[corr_names]
df_mice = mice_temp.filter(mice_names, axis=1).copy()
mice_estimator = IterativeImputer(estimator=linear_model.BayesianRidge(), sample_posterior=True, max_iter=40,
                                n_nearest_features=10, imputation_order='random')
df_mice_imputed = pd.DataFrame(mice_estimator.fit_transform(df_mice), columns=df_mice.columns)
df_mice_imputed = df_mice_imputed.apply(lambda x: x.abs(), axis=1)
imputed_mice = pd.concat([df_mice_imputed, temp_week], axis=1)

In [ ]:
imputed_mice.describe()

In [ ]:
KNN_imputation.describe()

### Section 6: Distribution Analysis of Imputed Data
This section is best used to compare if the top two distributions
and their parameters differ from the others. It can suggest the
imputation may be weak.

In [ ]:
temp_df = df.drop(['DATE'], axis=1)
temp_mice = imputed_mice.drop(['DATE'], axis=1)
temp_KNN = KNN_imputation.drop(['DATE'], axis=1)
distributions(temp_df)

In [ ]:
distributions(temp_mice)

In [ ]:
distributions(temp_KNN)

### Section 7: Imputation Distributions
This section shows the distributions for all of the variables and
can be used to determine how different they may be. The x-axis is
in dollars, and the density is a way to represent the mass under
the curve equally across variables with different properties.

In [ ]:
cols = 3
rows = 3
num_cols = df.select_dtypes(exclude='object').columns
fig = plt.figure(figsize=(cols * 3, rows * 3))
for i, col in enumerate(num_cols):
    ax = fig.add_subplot(rows, cols, i + 1)
    sns.kdeplot(data=imputed_mice[col])
fig.tight_layout()
plt.show()

In [ ]:
cols = 3
rows = 3
num_cols = df.select_dtypes(exclude='object').columns
fig = plt.figure(figsize=(cols * 3, rows * 3))
for i, col in enumerate(num_cols):
    ax = fig.add_subplot(rows, cols, i + 1)
    sns.kdeplot(data=KNN_imputation[col])
fig.tight_layout()
plt.show()

### Section 8: Original, KNN, and MICE Comparisons
If the data has no missing values, the following plot will look unremarkable,
but if data is imputed, three different curves will appear shifted, a useful
method to determine how different the imputed data is. The last two summaries
show the properties of the imputed data without the missing values.

In [ ]:
cols = 3
rows = 3
num_cols = df.select_dtypes(exclude='object').columns
fig = plt.figure(figsize=(10, 10))
for i, col in enumerate(num_cols):
    ax = fig.add_subplot(rows, cols, i + 1)
    sns.kdeplot(data=imputed_mice[col], color='blue', legend=True)
    sns.kdeplot(data=KNN_imputation[col], color='green', legend=True)
    sns.kdeplot(data=df[col], color='red', legend=True)
fig.tight_layout()
plt.show()

In [ ]:
summary(imputed_mice)

In [ ]:
summary(KNN_imputation)

### Section 9: RFE Feature Importance
This measure of feature importance is Recursive Feature Elimination. RFE uses an
estimator, in this case RandomForestRegressor, to eliminate variables one by one
until it meets a given threshold. This does not mean that the "Selected False"
variables do not add to the model, but a good portion of the model's accuracy
will remain even if they are dropped. It is possible to change the number of
features selected to reduce or increse the features, if necessary.

In [ ]:
X = df.drop(['DATE', 'revenue'], axis=1)
y = df['revenue']
rfe = RFE(RandomForestRegressor(n_estimators=500, criterion='squared_error', bootstrap=False), importance_getter='auto')
fit = rfe.fit(X, y)
for i in range(X.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
print(X.columns)

### Section 10: Cluster Evaluation
This method is another way to see the relationship between the dependent and independent
variables. Clusters are evaluated by similarity and provided with a silhouette score between
-1 and 1, with 1 meaning the clusters are very similar. The next two plots demonstrate
the dissimilarity of the clusters, and the last plot a score between the chosen features
and the dependent variable.

In [ ]:
df_no_date = df.drop('DATE', axis=1)

ce = clusteval(cluster='agglomerative', evaluate='silhouette', metric='euclidean', linkage='complete', min_clust=5, verbose=None)
results = ce.fit(df_no_date)
ce.plot()
ce.plot_silhouette(df_no_date)

In [ ]:
enrichment_results = ce.enrichment(df_no_date)
ce.scatter(n_feat=2, figsize=(14,14))

### Section 11: GridSearch
A GridSearch can be used to iterate through many combinations of features
to determine what the best combination is for the most accurate model. In this
case, the initial GridSearch model is built, its best model used for prediction
to determine a R-Squared score, then a feature importance rank is given. At this
point, most of the previous models and the following feature importance methods
should agree.

In [ ]:
X = df.drop(['DATE', 'revenue'], axis=1)
y = df['revenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.25)
cv = KFold(n_splits=5)
model = RandomForestRegressor()
param_search = {'n_estimators': [10, 50, 100]}
gs = GridSearchCV(estimator=model,
                  refit=True,
                  cv=cv,
                  param_grid=param_search)
gs.fit(X_train, y_train)
best_model = RandomForestRegressor(**gs.best_params_)
best_model.fit(X_train, y_train)
preds = best_model.predict(X_test)
performance = r2_score(y_test, preds)
print(performance)

In [ ]:
final_model = RandomForestRegressor(**gs.best_params_)
final_model.fit(X, y)
final_model.feature_importances_

In [ ]:
print(X.columns)

### Section 12: Feature Importance
This sections uses a RandomForestClassifier and a permutation importance to rank the most important
features. Ideally, this should match up with most or all of the previous feature assessments.

In [ ]:
X = df.drop(['DATE', 'revenue'], axis=1).astype(int)
y = df['revenue'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

forest = RandomForestClassifier(random_state=0)
forest.fit(X_train, y_train)
importances = forest.feature_importances_

rf = RandomForestClassifier(n_estimators=100, n_jobs=-1,
                            max_features=1.0,
                            min_samples_leaf=10, oob_score=True)
rf.fit(X_train, y_train)

I = pd.DataFrame()

I['Feature'] = X_train.columns
I['Importance'] = rf.feature_importances_
I = I.sort_values('Importance', ascending=False)
I = I.set_index('Feature')
plot_importances(I, width=16, vscale=4)

In [ ]:
perm_imp_rfpimp = permutation_importances(rf, X_train, y_train, r2)
plt.plot(perm_imp_rfpimp)
plot_importances(perm_imp_rfpimp, width=14, vscale=3)

### Section 13: Time Series Autocorrelation and Stationarity Tests
The plot provides a simple visualization to determine if autocorrelation exists. If
the line goes out of the bounds in the middle, autocorrelation exists. The second test,
the Augmented Dickey-Fuller Test, tests for stationarity and if differencing is required.
If the result is "True", it should require some degree of differencing.

In [ ]:
plt.rcParams.update({'figure.figsize':(10,4), 'figure.dpi':120})
autocorrelation_plot(imputed_mice['revenue'].tolist())

In [ ]:
plt.rcParams.update({'figure.figsize':(10,4), 'figure.dpi':120})
autocorrelation_plot(KNN_imputation['revenue'].tolist())

In [ ]:
mice_no_date = imputed_mice[['revenue']]
adf_test = ADFTest(alpha = .05)
adf_test.should_diff(mice_no_date['revenue'])

In [ ]:
knn_no_date = KNN_imputation[['revenue']]
adf_test = ADFTest(alpha = .05)
adf_test.should_diff(knn_no_date['revenue'])

### Section 14: Lag and Forecast Time Series Tests
This last section first tests whether there is lag between one observation and
the previous. Next, as an experiment, it performs an auto Arima and provides metrics
and the results.

In [ ]:
ac1 = df['revenue'].autocorr(lag=1)
print("One week Lag: ", ac1)
ac2 = df['revenue'].autocorr(lag=2)
print("Two week Lag: ", ac2)
ac3 = df['revenue'].autocorr(lag=3)
print("Three week Lag: ", ac3)
ac4 = df['revenue'].autocorr(lag=4)
print("Four Week Lag: ", ac4)
ac5 = df['revenue'].autocorr(lag=5)
print("Five Week Lag: ", ac5)
ac6 = df['revenue'].autocorr(lag=6)
print("Six Week Lag: ", ac6)

In [ ]:
train = df['revenue'][:72]
test = df['revenue'][100:]
model = auto_arima(train, trace=True, error_action='ignore', suppress_warnings=True)
model.fit(train)
forecast = model.predict(n_periods=len(test))
forecast = pd.DataFrame(forecast,index = test.index,columns=['Prediction'])

In [ ]:
forecast = forecast.squeeze()
forecast_accuracy(forecast, test)

In [ ]:
# df_small = df[['DATE', 'revenue']]
# result = adfuller(df_small.revenue.dropna())
# print('ADF Statistic: %f' % result[0])
# print('p-value: %f' % result[1])